In [2]:
import imp
import os
import shutil
import time
import pyrl.policygradient as PG
import numpy as np

In [3]:
import sys
sys.path.append('/home/hongli/Documents/pyrlearning/examples')
sys.path.append('/home/hongli/Documents/pyrlearning/pyrl')
sys.path.append('/home/hongli/Documents/pyrlearning/examples/models')


In [4]:
model_file = 'models/twoarmbandit'
action = 'train'
modelfile = os.path.abspath(model_file)
if not modelfile.endswith('.py'):
    modelfile += '.py'


In [5]:
# default settings
args = []
dt = 100
dt_save = 0
seed = 100
suffix = ''
gpu = False
print("MODELFILE: " + modelfile)
print("ACTION:    " + action)
print("ARGS:      " + str(args))
print("SEED:      " + str(seed))
print("SUFFIX:    " + suffix)
print("GPU:       " + str(gpu))

MODELFILE: /home/hongli/Documents/RLearning/models/twoarmbandit.py
ACTION:    train
ARGS:      []
SEED:      100
SUFFIX:    
GPU:       False


In [6]:
# Set flags before importing Theano
os.environ.setdefault('THEANO_FLAGS', '')
os.environ['THEANO_FLAGS'] += ',floatX=float32,allow_gc=False'
if gpu:
    os.environ['THEANO_FLAGS'] += ',device=gpu,nvcc.fastmath=True'

In [7]:
#=========================================================================================
# Setup paths
#=========================================================================================

from pyrl import utils
from pyrl.model import Model

# Location of script
# here   = utils.get_here(sys.argv[0])
# __file__ won't work in interactive interpretor 
here = utils.get_here('do.py') 
prefix = os.path.basename(here)

# Name to use
name = os.path.splitext(os.path.basename(modelfile))[0] + suffix

# Scratch
scratchpath = os.environ.get('SCRATCH')
if scratchpath is None:
    scratchpath = os.path.join(os.environ['HOME'], 'scratch')
trialspath = os.path.join(scratchpath, 'work', 'pyrl', prefix, name)

# Paths
workpath = os.path.join(here,     'work')
datapath = os.path.join(workpath, 'data', name)
figspath = os.path.join(workpath, 'figs', name)

# Create necessary directories
for path in [datapath, figspath, trialspath]:
    utils.mkdir_p(path)

# File to store model in
savefile = os.path.join(datapath, name + '.pkl')


In [8]:
model = Model(modelfile)

In [9]:
import inspect
inspect.getmembers(model)

[('Task', pyrl.model.Task),
 ('__class__', pyrl.model.Model),
 ('__delattr__',
  <method-wrapper '__delattr__' of Model object at 0x7fd6353ef610>),
 ('__dict__',
  {'Task': pyrl.model.Task,
   'config': {'L1_Wrec': 0,
    'L2_Wrec': 0,
    'L2_r': 0,
    'N': 100,
    'Nin': 2,
    'Nout': 2,
    'Performance': pyrl.performance.Performance2AFC,
    'R_ABORTED': 0,
    'R_TERMINAL': None,
    'Win': 1,
    'Win_mask': None,
    'abort_on_last_t': True,
    'actions': OrderedDict([('CHOOSE-LEFT', 0), ('CHOOSE-RIGHT', 1)]),
    'baseline_L2_r': 0,
    'baseline_N': 100,
    'baseline_Win': 10,
    'baseline_Win_mask': None,
    'baseline_bout': None,
    'baseline_fix': [],
    'baseline_lr': 0.004,
    'baseline_network_type': 'gru',
    'baseline_p0': 1,
    'baseline_rho': 2,
    'baseline_seed': 2,
    'baseline_var_rec': 0.01,
    'bout': 0.5,
    'checkfreq': 50,
    'dt': 10,
    'fix': [],
    'inputs': OrderedDict([('Choice', 0), ('Reward', 1)]),
    'lr': 0.004,
    'max_iter': 

In [10]:
#model.train(savefile, seed, recover=('recover' in args))
config = model.config
recover = False
Task = model.Task


In [11]:
print savefile
print seed
inspect.getmembers(Task)

/home/hongli/Documents/RLearning/work/data/twoarmbandit/twoarmbandit.pkl
100


[('__class__', type),
 ('__delattr__', <slot wrapper '__delattr__' of 'object' objects>),
 ('__dict__',
  dict_proxy({'__dict__': <attribute '__dict__' of 'Task' objects>,
              '__doc__': None,
              '__init__': <function pyrl.model.__init__>,
              '__module__': 'pyrl.model',
              '__weakref__': <attribute '__weakref__' of 'Task' objects>})),
 ('__doc__', None),
 ('__format__', <method '__format__' of 'object' objects>),
 ('__getattribute__', <slot wrapper '__getattribute__' of 'object' objects>),
 ('__hash__', <slot wrapper '__hash__' of 'object' objects>),
 ('__init__', <unbound method Task.__init__>),
 ('__module__', 'pyrl.model'),
 ('__new__', <function __new__>),
 ('__reduce__', <method '__reduce__' of 'object' objects>),
 ('__reduce_ex__', <method '__reduce_ex__' of 'object' objects>),
 ('__repr__', <slot wrapper '__repr__' of 'object' objects>),
 ('__setattr__', <slot wrapper '__setattr__' of 'object' objects>),
 ('__sizeof__', <method '__sizeo

In [13]:
# this part is the break down of inside of model.train...
if recover and os.path.isfile(savefile):
    pg = self.get_pg(savefile, load='current')  # get_pg returns a policygradient instance
else:
    config['seed']          = 3*seed
    config['policy_seed']   = 3*seed + 1
    config['baseline_seed'] = 3*seed + 2

    # pg = get_pg(config, config['seed'])
    pg = PG.PolicyGradient(Task, config, seed=seed, dt=config['dt'], load='best')

AttributeError: 'PolicyGradient' object has no attribute 'PolicyGradient'

In [14]:
# get policy gradient
# try to break down PolicyGradient setup
# this block is equivalent to "pg.PolicyGradient(task, config, seed=seed, dt=config['dt'], load='best')"
from pyrl.networks import Networks

task = Task()

# =================================================================================
# Network setup
# =================================================================================


#-----------------------------------------------------------------------------
# Create new model.
# #-----------------------------------------------------------------------------

# config comes from configs.py
# print "config is", config

# dt = 10ms
# Leak
alpha = dt/config['tau']

# Policy network
K = config['p0']*config['N']
policy_config = {
    'Nin':      config['Nin'],
    'N':        config['N'],
    'Nout':     config['Nout'],
    'p0':       config['p0'],
    'rho':      config['rho'],
    'f_out':    'softmax',
    'Win':      config['Win']*np.sqrt(K)/config['Nin'],
    'Win_mask': config['Win_mask'],
    'fix':      config['fix'],
    'L2_r':     config['L2_r'],
    'L1_Wrec':  config['L1_Wrec'],
    'L2_Wrec':  config['L2_Wrec'],
    'alpha':    alpha,
    'bout':     config['bout']
}

# print "bout:", self.policy_config['bout']

# Network type
Network = Networks[config['network_type']]
policy_net = Network(policy_config,
    seed=config['policy_seed'], name='policy')

# Baseline network
# Win = np.zeros((self.policy_net.N + len(config['actions']), 3*config['N']))
# Win[self.policy_net.N:] = 1
# what's baseline?

'''
            rng = np.random.RandomState(1234)
            policy_N     = config['N']
            baseline_N   = config['N']
            baseline_Nin = self.policy_net.N + len(config['actions'])
            baseline_Win_mask = np.zeros((baseline_Nin, 3*baseline_N))
            p_in = 0.5
            baseline_Win_mask[:policy_N] = (rng.uniform(size=baseline_Win_mask[:policy_N].shape) < p_in)
            #baseline_Win = 1/np.sqrt(p_in*baseline)
'''

K = config['baseline_p0']*config['N']
baseline_Nin = policy_net.N + len(config['actions'])
baseline_config = {
    'Nin':      baseline_Nin,
    'N':        config['baseline_N'],
    'Nout':     1,
    'p0':       config['baseline_p0'],
    'rho':      config['baseline_rho'],
    'f_out':    'linear',
    'Win':      config['baseline_Win']*np.sqrt(K)/baseline_Nin,
    'Win_mask': config['baseline_Win_mask'],
    'bout':     config['baseline_bout'],
    'fix':      config['baseline_fix'],
    'L2_r':     config['baseline_L2_r'],
    'L1_Wrec':  config['L1_Wrec'],
    'L2_Wrec':  config['L2_Wrec'],
    'alpha':    alpha
}


<< RNG 100 >> __main__


In [15]:
# PolicyGradient part 2

if baseline_config['bout'] is None:
    baseline_config['bout'] = config['R_ABORTED']

# Network type
Network = Networks[config.get('baseline_network_type',
                                    config['network_type'])]
baseline_net = Network(baseline_config,
                                    seed=config['baseline_seed'], name='baseline')

#=================================================================================
# PG setup
#=================================================================================

# Network structure
Nin  = config['Nin']
N    = config['N']
Nout = config['Nout']

# Number of actions
n_actions = len(config['actions'])

# Recurrent noise, scaled by `2*tau/dt`
scaled_var_rec = (2*config['tau']/dt) * config['var_rec']
scaled_baseline_var_rec = ((2*config['tau']/dt)
                                * config['baseline_var_rec'])

# Run trials continuously?
mode = config['mode']
if mode == 'continuous':
    step_0_states = policy_net.func_step_0(True)

# Maximum length of a trial
Tmax = int(config['tmax']/config['dt']) + 1

# Discount future reward
if np.isfinite(config['tau_reward']):
    alpha_reward = dt/config['tau_reward']
    def discount_factor(t):
        return np.exp(-t*alpha_reward)
else:
    def discount_factor(t):
        return 1
discount_factor = discount_factor

# Reward on aborted trials
abort_on_last_t = config.get('abort_on_last_t', True)
if 'R_TERMINAL' in config and config['R_TERMINAL'] is not None:
    R_TERMINAL = config['R_TERMINAL']
else:
    R_TERMINAL = config['R_ABORTED']
    R_ABORTED = config['R_ABORTED']


<< RNG 100 >> __main__


In [17]:

# Random number generator
import pyrl.nptools as nptools
rng = nptools.get_rng(seed, __name__)

# Compile functions
policy_step_0   = policy_net.func_step_0()
policy_step_t   = policy_net.func_step_t()
baseline_step_0 = baseline_net.func_step_0()
baseline_step_t = baseline_net.func_step_t()

# Performance
Performance = config['Performance']


<< RNG 100 >> __main__


In [26]:
# break down pg.train here
"""
Train network.

"""
import pyrl.theanotools as theanotools
from   collections import OrderedDict
from   theano import tensor
from pyrl.sgd import Adam
#=================================================================================
# Parameters
#=================================================================================

max_iter     = config['max_iter']
lr           = config['lr']
baseline_lr  = config['baseline_lr']
n_gradient   = config['n_gradient']
n_validation = config['n_validation']
checkfreq    = config['checkfreq']

if mode == 'continuous':
    print("[ PolicyGradient.train ] Continuous mode.")
    use_x0 = True
else:
    use_x0 = False

# GPU?
if theanotools.get_processor_type() == 'gpu':
    gpu = 'yes'
else:
    gpu = 'no'

# Print settings
items = OrderedDict()
items['GPU']                      = gpu
items['Network type (policy)']    = config['network_type']
items['Network type (baseline)']  = config.get('baseline_network_type',
                                                            config['network_type'])
items['N (policy)']               = config['N']
items['N (baseline)']             = config['baseline_N']
items['Conn. prob. (policy)']     = config['p0']
items['Conn. prob. (baseline)']   = config['baseline_p0']
items['dt']                       = str(dt) + ' ms'
items['tau_reward']               = str(config['tau_reward']) + ' ms'
items['var_rec (policy)']         = config['var_rec']
items['var_rec (baseline)']       = config['baseline_var_rec']
items['Learning rate (policy)']   = config['lr']
items['Learning rate (baseline)'] = config['baseline_lr']
items['Max time steps']           = Tmax
items['Num. trials (gradient)']   = config['n_gradient']
items['Num. trials (validation)'] = config['n_validation']


0

In [27]:
def func_update_policy(Tmax, use_x0=False, accumulators=None):
    """
    update policy network
    :param Tmax:
    :param use_x0:
    :param accumulators:
    :return:
    """
    U = tensor.tensor3('U') # Inputs
    Q = tensor.tensor3('Q') # Noise

    if use_x0:
        x0_ = tensor.matrix('x0_')
    else:
        x0  = policy_net.params['x0']
        x0_ = tensor.alloc(x0, U.shape[1], x0.shape[0])

    log_z_0  = policy_net.get_outputs_0(x0_, log=True)
    r, log_z = policy_net.get_outputs(U, Q, x0_, log=True)

    # Learning rate
    lr = tensor.scalar('lr')

    A = tensor.tensor3('A')
    R = tensor.matrix('R')
    b = tensor.matrix('b')
    M = tensor.matrix('M')

    logpi_0 = tensor.sum(log_z_0*A[0], axis=-1)*M[0]
    logpi_t = tensor.sum(log_z*A[1:],  axis=-1)*M[1:]

    # Entropy
    # entropy_0 = tensor.sum(tensor.exp(log_z_0)*log_z_0, axis=-1)*M[0]
    # entropy_t = tensor.sum(tensor.exp(log_z)*log_z, axis=-1)*M[1:]
    # entropy   = (tensor.sum(entropy_0) + tensor.sum(entropy_t))/tensor.sum(M)

    # def f(x):
    #    return -x**2/2/sigma**2

    # logpi_0 = tensor.sum(f(A[0] - z_0), axis=-1)*M[0]
    # logpi_t = tensor.sum(f(A[1:] - z), axis=-1)*M[1:]

    # Enforce causality
    Mcausal = theanotools.zeros((Tmax-1, Tmax-1))
    for i in xrange(Mcausal.shape[0]):
        Mcausal[i,i:] = 1
    Mcausal = theanotools.shared(Mcausal, 'Mcausal')

    J0 = logpi_0*R[0]
    J0 = tensor.mean(J0)
    J  = (logpi_t.T).dot(Mcausal).dot(R[1:]*M[1:])
    J  = tensor.nlinalg.trace(J)/J.shape[0]

    J += J0

    # Second term
    Jb0 = logpi_0*b[0]
    Jb0 = tensor.mean(Jb0)
    Jb  = logpi_t*b[1:]
    Jb  = tensor.mean(tensor.sum(Jb, axis=0))

    J -= Jb0 + Jb

    # Objective function
    obj = -J + policy_net.get_regs(x0_, r, M)# + 0.0005*entropy

    # SGD
    policy_sgd = Adam(policy_net.trainables, accumulators=accumulators)
    if policy_net.type == 'simple':
        i = policy_net.index('Wrec')
        grads = tensor.grad(obj, policy_net.trainables)
        grads[i] += policy_net.get_dOmega_dWrec(-J, r)
        norm, grads, updates = policy_sgd.get_updates(obj, lr, grads=grads)
    else:
        norm, grads, updates = policy_sgd.get_updates(obj, lr)

    if use_x0:
        args = [x0_]
    else:
        args = []
    args += [U, Q, A, R, b, M, lr]

    return theano.function(args, norm, updates=updates)

def func_update_baseline(self, use_x0=False, accumulators=None):
    """
    update value network
    the reward baseline is the output of a recurrently connected value work
    useful for learning but not executing
    expected reward
    :param use_x0:
    :param accumulators:
    :return:
    """
    U  = tensor.tensor3('U')
    R  = tensor.matrix('R')
    R_ = R.reshape((R.shape[0], R.shape[1], 1))
    Q  = tensor.tensor3('Q')

    if use_x0:
        x0_ = tensor.matrix('x0_')
    else:
        x0  = baseline_net.params['x0']
        x0_ = tensor.alloc(x0, U.shape[1], x0.shape[0])

    z_0   = baseline_net.get_outputs_0(x0_)
    r, z  = baseline_net.get_outputs(U, Q, x0_)
    z_all = tensor.concatenate([z_0.reshape((1, z_0.shape[0], z_0.shape[1])), z],
                                   axis=0)

    # Learning rate
    lr = tensor.scalar('lr')

    # Reward prediction error
    M    = tensor.matrix('M')
    L2   = tensor.sum((tensor.sqr(z_all[:,:,0] - R))*M)/tensor.sum(M)
    RMSE = tensor.sqrt(L2)

    # Objective function
    obj = L2 + baseline_net.get_regs(x0_, r, M)

    # SGD
    baseline_sgd = Adam(baseline_net.trainables, accumulators=accumulators)
    if baseline_net.type == 'simple':
        i = baseline_net.index('Wrec')
        grads = tensor.grad(obj, baseline_net.trainables)
        grads[i] += baseline_net.get_dOmega_dWrec(L2, r)
        norm, grads, updates = baseline_sgd.get_updates(obj, lr, grads=grads)
    else:
        norm, grads, updates = baseline_sgd.get_updates(obj, lr)

    if use_x0:
        args = [x0_]

    else:
        args = []
    args += [U, Q, R, M, lr]

    return theano.function(args, [z_all[:,:,0], norm, RMSE], updates=updates)

0

In [66]:

#=================================================================================
# Setup
#=================================================================================

#update_policy   = func_update_policy(Tmax, use_x0)

# let's break down func_update_policy here
"""
update policy network
:param Tmax:
:param use_x0:
:param accumulators:
:return:
"""
accumulators = None
U = tensor.tensor3('U') # Inputs
Q = tensor.tensor3('Q') # Noise

if use_x0:
    x0_ = tensor.matrix('x0_')
else:
    x0  = policy_net.params['x0']
    x0_ = tensor.alloc(x0, U.shape[1], x0.shape[0])

log_z_0  = policy_net.get_outputs_0(x0_, log=True)
r, log_z = policy_net.get_outputs(U, Q, x0_, log=True)

# Learning rate
lr = tensor.scalar('lr')

A = tensor.tensor3('A')
R = tensor.matrix('R')
b = tensor.matrix('b')
M = tensor.matrix('M')

logpi_0 = tensor.sum(log_z_0*A[0], axis=-1)*M[0]
logpi_t = tensor.sum(log_z*A[1:],  axis=-1)*M[1:]

print obj
print trainables
for x in trainables:
    print x.get_value()

Elemwise{add,no_inplace}.0
[Win, bin, Wrec_gates, Wrec, Wout, bout, x0]
[[ 5.89712731e-01  2.75748673e+00  5.61331714e-01  4.81714857e-01
   3.51221732e+00 -5.25251158e-01 -2.65258864e+00 -2.42043371e+00
   1.94426919e+00  1.30909400e+00  6.68499229e-01 -1.85455627e+00
  -2.02361209e+00  3.90004370e+00  4.42836822e-01 -4.74619394e-01
  -3.07157585e+00 -2.55827576e+00 -1.53741054e+00 -8.90633411e-02
  -1.40086121e+00 -1.24831680e+00 -2.54821933e+00 -9.73235731e-01
   1.13957379e+00  6.14914435e-01 -3.61713510e-01 -1.13549250e+00
   2.11210670e+00  6.58979620e-01 -2.72836021e-01  1.45464528e+00
  -2.00884598e+00 -1.17365406e+00  1.65145322e+00 -1.74988004e+00
   2.06998873e+00  5.71356138e-01 -2.63470722e+00 -1.01879056e+00
  -1.27141802e+00  4.19846474e-01 -1.00506500e+00  1.18373485e+00
   6.15645424e-01  1.02742268e+00  7.95002938e-01 -4.30618073e+00
   2.49418567e+00  2.30871982e-01  9.92137529e-01 -3.89853154e-01
  -2.30071069e+00 -6.60273529e-01 -1.01405497e+00 -7.01146499e-01
  -5

In [62]:
# Enforce causality
Mcausal = theanotools.zeros((Tmax-1, Tmax-1))
for i in xrange(Mcausal.shape[0]):
    Mcausal[i,i:] = 1
Mcausal = theanotools.shared(Mcausal, 'Mcausal')

J0 = logpi_0*R[0]
J0 = tensor.mean(J0)
J  = (logpi_t.T).dot(Mcausal).dot(R[1:]*M[1:])
J  = tensor.nlinalg.trace(J)/J.shape[0]

J += J0

# Second term
Jb0 = logpi_0*b[0]
Jb0 = tensor.mean(Jb0)
Jb  = logpi_t*b[1:]
Jb  = tensor.mean(tensor.sum(Jb, axis=0))

J -= Jb0 + Jb

# Objective function
obj = -J + policy_net.get_regs(x0_, r, M)# + 0.0005*entropy

policy_net.get_regs(x0_, r, M)

trainables = [policy_net.params[k]
                for k in policy_net.params if k not in policy_net.config['fix']]

trainables
policy_net.type

obj

Elemwise{add,no_inplace}.0

In [60]:
# SGD
policy_sgd = Adam(policy_net.trainables, accumulators=accumulators)
if policy_net.type == 'simple':
    i = policy_net.index('Wrec')
    grads = tensor.grad(obj, policy_net.trainables)
    grads[i] += policy_net.get_dOmega_dWrec(-J, r)
    norm, grads, updates = policy_sgd.get_updates(obj, lr, grads=grads)

else:
    # norm, grads, updates = policy_sgd.get_updates(obj, lr)
    # break get_dpdates here
    loss = obj
    grads = None
    # lr = lr
    if grads is None:
        grads = tensor.grad(loss, policy_sgd.trainables)

    # Clipping
    norm  = tensor.sqrt(sum([tensor.sqr(g).sum() for g in grads]))
    m     = theanotools.clipping_multiplier(norm, max_norm)
    grads = [m*g for g in grads]

    # Safeguard against numerical instability
    new_cond = tensor.or_(tensor.or_(tensor.isnan(norm), tensor.isinf(norm)),
                            tensor.or_(norm < 0, norm > 1e10))
    grads = [tensor.switch(new_cond, np.float32(0), g) for g in grads]

    # Safeguard against numerical instability
    #cond  = tensor.or_(norm < 0, tensor.or_(tensor.isnan(norm), tensor.isinf(norm)))
    #grads = [tensor.switch(cond, np.float32(0), g) for g in grads]

    # New values
    t       = policy_sgd.time + 1
    lr_t    = lr*tensor.sqrt(1. - beta2**t)/(1. - beta1**t)
    means_t = [beta1*m + (1. - beta1)*g for g, m in zip(grads, policy_sgd.means)]
    vars_t  = [beta2*v + (1. - beta2)*tensor.sqr(g) for g, v in zip(grads, policy_sgd.vars)]
    steps   = [lr_t*m_t/(tensor.sqrt(v_t) + epsilon)
                for m_t, v_t in zip(means_t, vars_t)]

    # Updates
    updates  = [(x, x - step) for x, step in zip(policy_sgd.trainables, steps)]
    updates += [(m, m_t) for m, m_t in zip(policy_sgd.means, means_t)]
    updates += [(v, v_t) for v, v_t in zip(policy_sgd.vars, vars_t)]
    updates += [(policy_sgd.time, t)]
    
if use_x0:
    args = [x0_]
else:
    args = []
args += [U, Q, A, R, b, M, lr]


update_policy = theano.function(args, norm, updates=updates)


DisconnectedInputError:  
Backtrace when that variable is created:

  File "/home/hongli/anaconda2/envs/scientific/lib/python2.7/site-packages/ipykernel/ipkernel.py", line 208, in do_execute
    res = shell.run_cell(code, store_history=store_history, silent=silent)
  File "/home/hongli/anaconda2/envs/scientific/lib/python2.7/site-packages/ipykernel/zmqshell.py", line 537, in run_cell
    return super(ZMQInteractiveShell, self).run_cell(*args, **kwargs)
  File "/home/hongli/anaconda2/envs/scientific/lib/python2.7/site-packages/IPython/core/interactiveshell.py", line 2714, in run_cell
    interactivity=interactivity, compiler=compiler, result=result)
  File "/home/hongli/anaconda2/envs/scientific/lib/python2.7/site-packages/IPython/core/interactiveshell.py", line 2818, in run_ast_nodes
    if self.run_code(code, result):
  File "/home/hongli/anaconda2/envs/scientific/lib/python2.7/site-packages/IPython/core/interactiveshell.py", line 2878, in run_code
    exec(code_obj, self.user_global_

In [ ]:
update_baseline = func_update_baseline(use_x0)

# Start training from here
iter_start = 0

# Keep track of best results
best_iter   = -1
best_reward = -np.inf
best_perf   = None
best_params = policy_net.get_values()
best_baseline_params = baseline_net.get_values()

# Initial states
init   = None
init_b = None
# Performance history
perf             = None
training_history = []
trials_tot       = 0


0

0

In [ ]:
#=================================================================================
# Train
#=================================================================================

        if hasattr(self.task, 'start_session'):
            self.task.start_session(self.rng)

        grad_norms_policy   = []
        grad_norms_baseline = []

        tstart = datetime.datetime.now()
        try:
            for iter_ in xrange(iter_start, max_iter+1):
                if iter_ % checkfreq == 0 or iter_ == max_iter:
                    if hasattr(self.task, 'n_validation'):
                        n_validation = self.task.n_validation
                    if n_validation > 0:
                        #-----------------------------------------------------------------
                        # Validation
                        #-----------------------------------------------------------------

                        # Report
                        elapsed = utils.elapsed_time(tstart)
                        print("After {} updates ({})".format(iter_, elapsed))

                        # RNG state
                        rng_state = self.rng.get_state()

                        # Trials

                        #for bandit task, generate blockID randomly every batch


                        n = 0
                        iter = 0
                        blockID = []
                        while n < n_validation:
                            blocklength = random.randint(20,50)
                            for x in range(blocklength):
                                blockID.append(iter%2)
                            n += blocklength
                            iter += 1


                        trials = [self.task.get_condition(self.rng, self.dt, blockID[i])
                                   for i in xrange(n_validation)]
                        #trials = n_validation
                        #
                        (U, Q, Q_b, Z, Z_b, A, R, M, init_, init_b_, x0_, x0_b_,
                         perf_) = self.run_trials(trials, progress_bar=True)
                        if hasattr(self.task, 'update'):
                            self.task.update(perf_)

                        # Termination condition
                        terminate = False
                        if hasattr(self.task, 'terminate'):
                            if self.task.terminate(perf_):
                                terminate = True

                        # Save
                        mean_reward = np.sum(R*M)/n_validation
                        record = {
                            'iter':        iter_,
                            'mean_reward': mean_reward,
                            'n_trials':    trials_tot,
                            'perf':        perf_
                            }
                        if mean_reward > best_reward or terminate:
                            best_iter   = iter_
                            best_reward = mean_reward
                            best_perf   = perf_
                            best_params          = self.policy_net.get_values()
                            best_baseline_params = self.baseline_net.get_values()

                            record['new_best'] = True
                            training_history.append(record)
                        else:
                            record['new_best'] = False
                            training_history.append(record)

                        # Save
                        save = {
                            'iter':                    iter_,
                            'config':                  self.config,
                            'policy_config':           self.policy_net.config,
                            'baseline_config':         self.baseline_net.config,
                            'policy_masks':            self.policy_net.get_masks(),
                            'baseline_masks':          self.baseline_net.get_masks(),
                            'current_policy_params':   self.policy_net.get_values(),
                            'current_baseline_params': self.baseline_net.get_values(),
                            'best_iter':               best_iter,
                            'best_reward':             best_reward,
                            'best_perf':               best_perf,
                            'best_policy_params':      best_params,
                            'best_baseline_params':    best_baseline_params,
                            'rng_state':               rng_state,
                            'init':                    init,
                            'init_b':                  init_b,
                            'perf':                    perf,
                            'training_history':        training_history,
                            'trials_tot':              trials_tot,
                            'net_sgd':                 self.policy_sgd.get_values(),
                            'baseline_sgd':            self.baseline_sgd.get_values()
                            }
                        utils.save(savefile, save)

                        # Reward
                        items = OrderedDict()
                        items['Best reward'] = '{} (iteration {})'.format(best_reward,
                                                                          best_iter)
                        items['Mean reward'] = '{}'.format(mean_reward)

                        # Performance
                        if perf_ is not None:
                            items.update(perf_.display(output=False))

                        # Value prediction error
                        V = np.zeros_like(R)
                        for k in xrange(V.shape[0]):
                            V[k] = np.sum(R[k:]*M[k:], axis=0)
                        error = np.sqrt(np.sum((Z_b - V)**2*M)/np.sum(M))
                        items['Prediction error'] = '{}'.format(error)

                        # Gradient norms
                        if len(grad_norms_policy) > 0:
                            if DEBUG:
                                items['|grad| (policy)']   = [len(grad_norms_policy)] + [f(grad_norms_policy)
                                                              for f in [np.min, np.median, np.max]]
                                items['|grad| (baseline)'] = [len(grad_norms_baseline)] + [f(grad_norms_baseline)
                                                              for f in [np.min, np.median, np.max]]
                            grad_norms_policy   = []
                            grad_norms_baseline = []

                        # Print
                        utils.print_dict(items)

                        # Target reward reached
                        if best_reward >= self.config['target_reward']:
                            print("Target reward reached.")
                            return

                        # Terminate
                        if terminate:
                            print("Termination criterion satisfied.")
                            return
                    else:
                        '''
                        #-----------------------------------------------------------------
                        # Ongoing learning
                        #-----------------------------------------------------------------

                        if not training_history:
                            training_history.append(perf)
                        if training_history[0] is None:
                            training_history[0] = perf

                        # Save
                        save = {
                            'iter':                    iter,
                            'config':                  self.config,
                            'policy_config':           self.policy_net.config,
                            'baseline_config':         self.baseline_net.config,
                            'masks_p':                 self.policy_net.get_masks(),
                            'masks_b':                 self.baseline_net.get_masks(),
                            'current_policy_params':   self.policy_net.get_values(),
                            'current_baseline_params': self.baseline_net.get_values(),
                            'rng_state':               self.rng.get_state(),
                            'init':                    init,
                            'init_b':                  init_b,
                            'perf':                    perf,
                            'training_history':        training_history,
                            'trials_tot':              trials_tot,
                            'net_sgd':                 self.policy_sgd.get_values(),
                            'baseline_sgd':            self.baseline_sgd.get_values()
                            }
                        utils.save(savefile, save)
                        '''
                        if perf is not None:
                            perf.display()

                        # Termination condition
                        terminate = False
                        if hasattr(self.task, 'terminate'):
                            if perf is not None and self.task.terminate(perf):
                                terminate = True
                        '''
                        # Report
                        if iter % 100 == 1:
                            elapsed = utils.elapsed_time(tstart)
                            print("After {} updates ({})".format(iter, elapsed))
                            if perf is not None:
                                perf.display()
                        '''
                        # Terminate
                        if terminate:
                            print("Termination criterion satisfied.")
                            return

                if iter_ == max_iter:
                    print("Reached maximum number of iterations ({}).".format(iter_))
                    sys.exit(0)

                #-------------------------------------------------------------------------
                # Run trials
                #-------------------------------------------------------------------------

                # Trial conditions
                if hasattr(self.task, 'n_gradient'):
                    n_gradient = self.task.n_gradient

                # for bandit task, generate blockID randomly every batch

                n = 0
                iter = 0
                blockID_grad = []
                while n < n_gradient:
                    blocklength = random.randint(20, 50)
                    for x in range(blocklength):
                        blockID_grad.append(iter % 2)
                    n += blocklength
                    iter += 1

                trials = [self.task.get_condition(self.rng, self.dt, blockID_grad[i])
                          for i in xrange(n_gradient)]

                # Run trials
                (U, Q, Q_b, Z, Z_b, A, R, M, init, init_b, x0, x0_b,
                 perf, r_policy, r_value) = self.run_trials(trials,
                                                            init=init, init_b=init_b,
                                                            return_states=True, perf=perf)

                #-------------------------------------------------------------------------
                # Update baseline
                #-------------------------------------------------------------------------

                baseline_inputs = np.concatenate((r_policy, A), axis=-1)

                # Compute return
                R_b = np.zeros_like(R)
                for k in xrange(R.shape[0]):
                    R_b[k] = np.sum(R[k:]*M[k:], axis=0)

                if use_x0:
                    args = [x0_b]
                else:
                    args = []
                args += [baseline_inputs[:-1], Q_b, R_b, M, baseline_lr]
                b, norm_b, rmse = update_baseline(*args)
                #print("Prediction error = {}".format(rmse))

                norm_b = float(norm_b)
                #print("norm_b = {}".format(norm_b))
                if np.isfinite(norm_b):
                    grad_norms_baseline.append(float(norm_b))

                #-------------------------------------------------------------------------
                # Update policy
                #-------------------------------------------------------------------------

                if use_x0:
                    args = [x0]
                else:
                    args = []
                args += [U[:-1], Q, A, R, b, M, lr]
                norm = update_policy(*args)

                norm = float(norm)
                #print("norm = {}".format(norm))
                if np.isfinite(norm):
                    grad_norms_policy.append(norm)

                trials_tot += n_gradient

        except KeyboardInterrupt:
            print("Training interrupted by user during iteration {}.".format(iter_))
            sys.exit(0)


0

In [48]:
Network

pyrl.gru.GRU